# Variables

In [10]:
# the path that points to a subfolder where the json files will be strored
# it is essential that the same value with the same value is used by the transformnotebook
filePath = "Files/PowerBIAndFabricSummit/2024/"

StatementMeta(, b7832cb6-6fb5-431e-aba8-a05ad1e36071, 12, Finished, Available)

# Imports
Importing all packages and functions that will be used during the extraction of the source data

In [1]:
import json
import requests
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import explode

StatementMeta(, ccfba773-3e92-46d3-8757-b6281dba4aa5, 3, Finished, Available)

# Get initial data
+ agenda
+ timetables
+ rooms
+ schedule

## get agenda all and write json to file section

In [12]:

uriWebSite = "https://powerbisummitapp.azurewebsites.net/api/Session/All"
header = ""
normalized_data = dict()
urlEndpoint_response = requests.get(f"{uriWebSite}", headers = header).json()
urlEndpoint_response_json = requests.get(f"{uriWebSite}", headers = header).json()['sessions']
#print(urlEndpoint_response_json)

urlEndpoint_response_json_dumps = json.dumps(urlEndpoint_response_json)
normalized_data["_data"] = urlEndpoint_response_json_dumps
a = json.dumps( normalized_data["_data"] )
payload = json.loads(a)
payload_rdd = spark.sparkContext.parallelize([payload])

# create the dataframe
df_agenda = spark.read \
    .option("inferSchema", "true") \
    .option("multiline","true") \
    .json(payload_rdd)

#display(df_agenda.head(10))

StatementMeta(, b7832cb6-6fb5-431e-aba8-a05ad1e36071, 14, Finished, Available)

In [13]:
df_agenda.write \
    .option("multiline", "true") \
    .mode("overwrite") \
    .json(f"{filePath}agenda.json")

StatementMeta(, b7832cb6-6fb5-431e-aba8-a05ad1e36071, 15, Finished, Available)

## timetable, rooms and session schedule

In [14]:
uriWebSite = "https://powerbisummitapp.azurewebsites.net/api/Session"
header = ""

timetable_normalized_data = dict()
room_normalized_data = dict()
schedule_normalized_data = dict()

urlEndpoint_response = requests.get(f"{uriWebSite}", headers = header)

timetable_json = urlEndpoint_response.json()['timeTable']
room_json = urlEndpoint_response.json()['room']
schedule_json = urlEndpoint_response.json()['schedule']

timetable_json_dumps = json.dumps(timetable_json)
timetable_normalized_data["_data"] = timetable_json_dumps

room_json_dumps = json.dumps(room_json)
room_normalized_data["_data"] = room_json_dumps

schedule_json_dumps = json.dumps(schedule_json)
schedule_normalized_data["_data"] = schedule_json_dumps

a = json.dumps( timetable_normalized_data["_data"] )
timetable_payload = json.loads(a)
timetable_payload_rdd = spark.sparkContext.parallelize([timetable_payload])

a = json.dumps( room_normalized_data["_data"] )
room_payload = json.loads(a)
room_payload_rdd = spark.sparkContext.parallelize([room_payload])

a = json.dumps( schedule_normalized_data["_data"] )
schedule_payload = json.loads(a)
schedule_payload_rdd = spark.sparkContext.parallelize([schedule_payload])

# create the dataframes
df_timetable = spark.read \
    .option("inferSchema", "true") \
    .option("multiline","true") \
    .json(timetable_payload_rdd)

df_room = spark.read \
    .option("inferSchema", "true") \
    .option("multiline","true") \
    .json(room_payload_rdd)

df_schedule = spark.read \
    .option("inferSchema", "true") \
    .option("multiline","true") \
    .json(schedule_payload_rdd)

#display(df_timetable.head(10))
#display(df_room.head(10))
#display(df_schedule.head(10))

StatementMeta(, b7832cb6-6fb5-431e-aba8-a05ad1e36071, 16, Finished, Available)

In [15]:
df_timetable.write \
    .option("multiline", "true") \
    .mode("overwrite") \
    .json(f"{filePath}timetable.json")

df_room .write \
    .option("multiline", "true") \
    .mode("overwrite") \
    .json(f"{filePath}room.json")

df_schedule.write \
    .option("multiline", "true") \
    .mode("overwrite") \
    .json(f"{filePath}schedule.json")

StatementMeta(, b7832cb6-6fb5-431e-aba8-a05ad1e36071, 17, Finished, Available)